# Projet Musculation

 L'objectif est d'avoir des fonctionnalités utiles pour évaluer la force en musculation

In [1]:
# importation packages

import numpy as np
import pandas as pd

En musculation, il y a trois grands axes de performances : 
* ***La force*** : de 1 à 5 répétitions
* ***L'endurance de force*** : de 5 à 12 répétitions 
* ***l'endurance*** : plus de 12 répétitions

Pour comparer les performances entre elles, un élément de comparaison est la répétition maximale. Autrement dit, la masse (appelée vulgairement poids) maximale soulevée en une répetition. Elle sert aussi à déterminer la progression. 

La référence de base pour le calcul de la 1 RM théorique est la table de Berger 

In [2]:
# Création de la table de Berger en DataFrame

rep_data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12,16, 20, 22, 25]
prc_data = [100, 97.4, 94.9, 92.5, 89.8, 87.6, 85.5, 83.3, 81.1, 78.8, 70, 65, 60, 55, 50]

df = pd.DataFrame({'rep_data': rep_data, 'prc_data': prc_data})

df

,rep_data,prc_data
0,1,100.0
1,2,97.4
2,3,94.9
3,4,92.5
4,5,89.8
5,6,87.6
6,7,85.5
7,8,83.3
8,9,81.1
9,10,78.8


In [3]:
#Calcul de la répetition Max à partir de la table de Berger et de la formule d'Epley

def calcul_1rm(rep, kg):
    if rep in df['rep_data'].values:
        index_of_rep = df.index[df['rep_data'] == rep].tolist()[0]
        kg_max = kg * 100 / df.loc[index_of_rep, 'prc_data']
        
    else :
        kg_max=  (kg * rep/30) + kg #formule d'Epley
        
    return kg_max

In [29]:
# Correspondance de la charge soulevée théorique dans la table de Berger

def correspondance(rep, kg):
    df_corr = df.copy()

    if rep in df_corr['rep_data'].values:
        kg_max= calcul_1rm(rep, kg)
        
        for i in range(df_corr.shape[0]):
            kg_th =  round(df_corr.loc[i, 'prc_data'] * kg_max / 100, 2)
            df_corr.loc[i, 'Kg_Th_data'] = kg_th
    
    else :
        kg_max = (kg * rep / 30) + kg  # Formule d'Epley
        df_corr = correspondance(1, kg_max)
        new_prc=round(100*kg/kg_max,1)
        new_rep = {'rep_data': rep, 'prc_data': new_prc, 'Kg_Th_data': kg}
        df_corr = pd.concat([df_corr, pd.DataFrame([new_rep])], ignore_index=True)
        df_corr = df_corr.sort_values(by='rep_data', ignore_index=True)

    return df_corr


In [30]:
correspondance(10,30)

,rep_data,prc_data,Kg_Th_data
0,1,100.0,38.07
1,2,97.4,37.08
2,3,94.9,36.13
3,4,92.5,35.22
4,5,89.8,34.19
5,6,87.6,33.35
6,7,85.5,32.55
7,8,83.3,31.71
8,9,81.1,30.88
9,10,78.8,30.00


In [31]:
correspondance(13,30)

,rep_data,prc_data,Kg_Th_data
0,1,100.0,43.00
1,2,97.4,41.88
2,3,94.9,40.81
3,4,92.5,39.78
4,5,89.8,38.61
5,6,87.6,37.67
6,7,85.5,36.76
7,8,83.3,35.82
8,9,81.1,34.87
9,10,78.8,33.88


In [32]:
calcul_1rm(13,30)

43.0

Dans le cadre de la progression continue, l'objectif est d'améliorer les performances de séances en séances. Pour cela, il faut connaître l'équivalent théorique.

*Par exemple, si Lundi sur un développé couché haltère, la performance de 10 répetitions à 35 kg est atteinte. Et que l'objectif est de rester sur un nombre de répetitions inférieur ou égal à 10. Alors à la séance suivante, il faut passer à 8 répetitions à 37 kg pour ensuite à la séance d'après réussir à faire 9 répetitions à 37 kg ce qui est une augmentation de force ( la 1 RM théorique sera donc augmentée)*

In [47]:
# calcul equivalent théorique 
def equivalent_th(rep, kg, rep_th):
    df_corr = correspondance(rep, kg)
    index_rep_th = df_corr[df_corr['rep_data'] == rep_th+1].index[0]
    
    if index_rep_th == 0:
        # Si l'index de rep_th est 0, cela signifie qu'il n'y a pas de ligne précédente, donc on retourne la première ligne
        kg_th = df_corr.iloc[0]
    else:
        # Sinon, on décale l'index d'une position vers le haut pour obtenir la ligne précédente
        kg_th = df_corr.iloc[index_rep_th - 1]
    
    return kg_th

In [48]:
equivalent_th(10,35,8)

rep_data       8.0
prc_data      83.3
Kg_Th_data    37.0
Name: 7, dtype: float64

#### Évaluation du niveau selon la 1rm

La référence en la matière provient des travaux de Charles Poliquin, un entraineur de force canadien. 

In [49]:
# création des matrices de coefficients à partir des connaissances de Charles Poliquin.

force_homme_coef_1rm = np.array([[105, 141, 174, 210, 236, 263, 315],
                                [79, 106, 131, 158, 178, 198, 237],
                                [132, 176, 218, 263, 296, 329, 395],
                                [53, 70, 87, 105, 118, 131, 158],
                                [90, 120, 149, 179, 202, 224, 269]])

force_homme_coef_6rm = np.array([[44, 59, 73, 88, 99, 110, 132],
                                [56, 74, 92, 111, 125, 139, 167], 
                                [67, 90, 111, 134, 151, 168, 201], 
                                [75, 101, 125, 151, 170, 188, 226],
                                [-16, 12, 39, 68, 88, 109, 151], 
                                [61, 82, 101, 122, 137, 153, 183], 
                                [28, 38, 47, 57, 64, 71, 85], 
                                [40, 54, 67, 81, 91, 101, 121], 
                                [19, 25, 31, 37, 42, 47, 56], 
                                [-33, -10, 11, 34, 51, 68, 101], 
                                [56, 74, 92, 111, 125, 139, 167], 
                                [28, 38, 47, 57, 64, 71, 85], 
                                [32, 42, 52, 63, 71, 79, 95],
                                [16, 21, 26, 32, 36, 39, 47],
                                [13, 18, 22, 27, 30, 33, 40],
                                [27, 36, 44, 53, 60, 67, 80], 
                                [21, 29, 35, 43, 48, 53, 64]])


force_femme_coef_1rm = np.array([[84, 113, 139, 168, 189, 210, 252],
                                 [63, 85, 105, 126, 142, 158, 190],
                                 [105, 141, 175, 210, 237, 263, 316],
                                 [42, 56, 70, 84, 95, 105, 126],
                                 [72, 96, 119, 144, 162, 179, 215]])

force_femme_coef_6rm = np.array([[35, 47, 58, 70, 79, 88, 106],
                                 [44, 59, 74, 89, 100, 111, 133],
                                 [54, 72, 89, 107, 121, 134, 161],
                                 [60, 81, 100, 121, 136, 151, 181],
                                 [-13, 10, 31, 54, 71, 88, 121],
                                 [49, 65, 81, 98, 110, 122, 147],
                                 [23, 31, 38, 46, 51, 57, 68],
                                 [32, 43, 54, 64, 73, 81, 97],
                                 [15, 20, 25, 30, 34, 37, 45],
                                 [-26, -8, 9, 27, 41, 54, 81],
                                 [44, 59, 74, 89, 100, 111, 133],
                                 [23, 30, 38, 45, 51, 57, 68],
                                 [25, 34, 42, 51, 57, 63, 76],
                                 [13, 17, 21, 25, 28, 32, 38],
                                 [11, 14, 18, 21, 24, 27, 32],
                                 [21, 29, 35, 43, 48, 53, 64],
                                 [17, 23, 28, 34, 38, 43, 51]])


# Dataframe des performances selon le poids

# Noms des lignes et colonnes
lignes_1Rm = ["Squat 1Rm", "Développé Couché 1Rm", "Soulevé de Terre 1Rm", "Développé Militaire 1Rm", "Squat Avant 1Rm"]
lignes_6Rm = ["Développé militaire 6Rm",
              "Développé incliné 6Rm",
              "Développé couché 6Rm",
              "Développé décliné 6Rm",
              "Dips, poids du lest 6Rm",
              "Dpé couché serré, env. 40 cm, 6Rm",
              "Développé couché haltères 6Rm",
              "Développé nuque assis 6Rm",
              "Développé haltères assis 6Rm",
              "Tractions supi., poids du lest 6Rm",
              "Rowing barre 90° large 6Rm",
              "Barre front non triché 6Rm",
              "Curl barre non triché 6Rm",
              "Curl haltères alterné 6Rm",
              "Curl haltères incliné 45° 6Rm",
              "Curl pupitre 6Rm",
              "Curl inversé 6Rm"]



colonnes = ["Novice", "Débutant", "Intermédiaire", "Confirmé", "Avancé", "Expert", "Elite"]

# Création des DataFrame
df_force_homme_coef_1rm = pd.DataFrame(data=force_homme_coef_1rm*0.01, index=lignes_1Rm , columns=colonnes)
df_force_homme_coef_6rm = pd.DataFrame(data=force_homme_coef_6rm*0.01, index=lignes_6Rm , columns=colonnes)

df_force_femme_coef_1rm = pd.DataFrame(data=force_femme_coef_1rm*0.01, index=lignes_1Rm , columns=colonnes)
df_force_femme_coef_6rm = pd.DataFrame(data=force_femme_coef_6rm*0.01, index=lignes_6Rm , columns=colonnes)






In [50]:
df_force_femme_coef_1rm

,Novice,Débutant,Intermédiaire,Confirmé,Avancé,Expert,Elite
Squat 1Rm,0.84,1.13,1.39,1.68,1.89,2.10,2.52
Développé Couché 1Rm,0.63,0.85,1.05,1.26,1.42,1.58,1.90
Soulevé de Terre 1Rm,1.05,1.41,1.75,2.10,2.37,2.63,3.16
Développé Militaire 1Rm,0.42,0.56,0.70,0.84,0.95,1.05,1.26
Squat Avant 1Rm,0.72,0.96,1.19,1.44,1.62,1.79,2.15


In [51]:
df_force_femme_coef_6rm

,Novice,Débutant,Intermédiaire,Confirmé,Avancé,Expert,Elite
Développé militaire 6Rm,0.35,0.47,0.58,0.70,0.79,0.88,1.06
Développé incliné 6Rm,0.44,0.59,0.74,0.89,1.00,1.11,1.33
Développé couché 6Rm,0.54,0.72,0.89,1.07,1.21,1.34,1.61
Développé décliné 6Rm,0.60,0.81,1.00,1.21,1.36,1.51,1.81
"Dips, poids du lest 6Rm",-0.13,0.10,0.31,0.54,0.71,0.88,1.21
"Dpé couché serré, env. 40 cm, 6Rm",0.49,0.65,0.81,0.98,1.10,1.22,1.47
Développé couché haltères 6Rm,0.23,0.31,0.38,0.46,0.51,0.57,0.68
Développé nuque assis 6Rm,0.32,0.43,0.54,0.64,0.73,0.81,0.97
Développé haltères assis 6Rm,0.15,0.20,0.25,0.30,0.34,0.37,0.45
"Tractions supi., poids du lest 6Rm",-0.26,-0.08,0.09,0.27,0.41,0.54,0.81


In [ ]:
# Calcul de la force athlétique sans dopage

def niveau_force_df(poids, sexe):
    if sexe =='F':
        df_force_1rm=poids*df_force_femme_coef_1rm
        df_force_6rm=poids*df_force_femme_coef_6rm
        return df_force_1rm, df_force_6rm
    elif sexe =='M' :
        df_force_1rm=poids*df_force_homme_coef_1rm
        df_force_6rm=poids*df_force_homme_coef_6rm
        return df_force_1rm, df_force_6rm
    else:
        print("Erreur ! sexe = M si homme, sexe = F si femme")
    


In [38]:
# Exemple d'appel de la fonction pour un homme avec un poids de 75 kg
poids_test = 75
sexe_test = 'M'
df_force_1rm, df_force_6rm = niveau_force_df(poids_test, sexe_test)

# Affichage des DataFrames
print("Force 1RM :")
print(df_force_1rm)

print("\nForce 6RM :")
print(df_force_6rm)

Force 1RM :
                         Novice  Débutant  Intermédiaire  Confirmé  Avancé  \
Squat 1Rm                 78.75    105.75         130.50    157.50   177.0   
Développé Couché 1Rm      59.25     79.50          98.25    118.50   133.5   
Soulevé de Terre 1Rm      99.00    132.00         163.50    197.25   222.0   
Développé Militaire 1Rm   39.75     52.50          65.25     78.75    88.5   
Squat Avant 1Rm           67.50     90.00         111.75    134.25   151.5   

                         Expert   Elite  
Squat 1Rm                197.25  236.25  
Développé Couché 1Rm     148.50  177.75  
Soulevé de Terre 1Rm     246.75  296.25  
Développé Militaire 1Rm   98.25  118.50  
Squat Avant 1Rm          168.00  201.75  

Force 6RM :
                                    Novice  Débutant  Intermédiaire  Confirmé  \
Développé militaire 6Rm              33.00     44.25          54.75     66.00   
Développé incliné 6Rm                42.00     55.50          69.00     83.25   
Développé c